# Multivariable Linear Regression

- 기존의 선형회귀는 x가 하나일 때 처리한 방식이다. 
- 그렇다면 여러 x개 있는 경우에는 어떻게 계산될까?

In [3]:
import numpy as np
import torch

x_train = torch.FloatTensor([[73, 80, 75],
[93, 88, 93],
[89, 91, 90],
[96, 98, 100],
[73, 66, 70]])

y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## Hypothesis

- 기존에는 `H(x) = wx + b`였다
- 이번에는 `H(x) = w1x1 + w2x2 + w3x3 + b`로 계산을 할 것 이다. 
- 문제는, 가중치를 위처럼 정적으로 계산하게 한다면 x가 길이 n인 벡터일 때, w도 n개를 수동으로 초기화시켜야 한다. 
- for문을 써도 매한가지다. 
- 이럴 때 쓰려고 `matmul`을 배웠다

In [4]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = x_train.matmul(W) + b

optimizer = torch.optim.SGD([W, b], lr=1e-5)

- 위에서 optim까지 정의했다면, epoch를 이용해 hypothesis를 계산한다. 
- Cost를 계산해 경사하강법으로 W, b를 예측한다. 

In [7]:
nb_epochs = 20

for epoch in range(nb_epochs+1):
    hypothesis = x_train.matmul(W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d} / {} hypothesis : {} Cost : {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0 / 20 hypothesis : tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost : 9298.520508
Epoch    1 / 20 hypothesis : tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost : 2915.712402
Epoch    2 / 20 hypothesis : tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost : 915.040527
Epoch    3 / 20 hypothesis : tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost : 287.936096
Epoch    4 / 20 hypothesis : tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost : 91.371063
Epoch    5 / 20 hypothesis : tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost : 29.758249
Epoch    6 / 20 hypothesis : tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost : 10.445267
Epoch    7 / 20 hypothesis : tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost : 4.391237
Epoch    8 / 20 hypothesis : tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost : 2.493121
Epoch    9 / 20 hypothesis : tensor([152.3454, 183.1161,

- 위에서 detach는 Pytorch의 Autograd에서 해당 텐서를 그래프에서 제거하는 역할을 한다. 
- detach를 하지 않으면 해당 텐서를 이용해 그래프를 추적할 수 있게 된다. 
- 이를 방지하고, 출력 및 저장용으로 특정 텐서를 저장하기 위해 detach를 많이 사용한다. 
- 그 외에도 `tensor.detach().cpu().numpy()`의 방식으로 사용하기 도 한다. 
	- detach() : 학습 그래프에서 분리
	- cpu() : GPU 텐서를 CPU로 이동
	- numpy() : 넘파이 배열로 변환